### Load libraries

In [1]:
from datetime import datetime, time, timedelta
import random
from math import copysign
from time import sleep
import sys
import numpy as np
import pandas as pd
from xtquant.xtdata import subscribe_quote, get_market_data, unsubscribe_quote, timetag_to_datetime,get_trade_times
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
from xtquant import xtconstant

from myPreparedata import list2csv, func1, account_type_num2str, account_status_num2str, \
    order_status_num2str, get_stock_price, func2, compact_status_num2str
from mytrade1 import MyXtQuantTrader, MyXtQuantTraderCallback,get_account_status,on_data, \
    get_order_details,get_trade_details,get_asset_details,get_position_details,get_credit_account_details,\
    get_compact_details,long_portfolio,short_portfolio,get_qty

### Register Account

In [2]:
session_id = random.randint(0, 10000)
xt_trader = MyXtQuantTrader(session_id)
callback = MyXtQuantTraderCallback()
xt_trader.register_callback(callback)
acc = StockAccount('140080060269','CREDIT')

### Start Trade thead.

In [3]:
xt_trader.start()
connect_result = xt_trader.connect()
if connect_result != 0:
    import sys
    sys.exit('链接失败，程序即将退出 %d'%connect_result)
status = xt_trader.subscribe(acc)
print("=" * 40)
print(f"账号订阅:{status}")
# 反订阅账号信息
# status = xt_trader.unsubscribe(acc)
# print(f"账号反订阅:{status}")
get_account_status(xt_trader, syncFlag=False)

账号订阅:0


In [4]:
ret=get_trade_times('SH')
print(ret)

[]
140080060269 CREDIT_ACCOUNT ACCOUNT_STATUS_OK
140000060269 SECURITY_ACCOUNT ACCOUNT_STATUS_OK


### Subscribe/Unsubsribe Single Quote.

In [5]:
symbols=['510350.SH','510330.SH']
for stock_code in symbols:
    seq=subscribe_quote(stock_code, period='1m', start_time='', end_time='', count=0, callback=on_data)
    if seq>0:
        print(f"订阅{stock_code}成功。")
    else:
        print(f"订阅{stock_code}失败。seq:{seq}")
unsubscribe_quote(seq)

订阅510350.SH成功。
订阅510330.SH成功。


### Query Order, Trade, Asset, Positions, Credit Account,Compacts

In [6]:
"""ORDER"""
orders  = xt_trader.query_stock_orders(acc,  cancelable_only = False)
if orders :
    order_details=get_order_details(orders)
    display(order_details)
"""TRADE"""
trades = xt_trader.query_stock_trades(acc)
if trades:
    trade_details=get_trade_details(trades)
    display(trade_details)
"""ASSET"""
assets = xt_trader.query_stock_asset(acc)
if assets:
    asset_detail=get_asset_details(assets)
    display(asset_detail)
"""POSITIONS"""
positions = xt_trader.query_stock_positions(acc)
if len(positions) != 0:
    position_details=get_position_details(positions)
    display(position_details)
"""CREDIT ACCOUNT"""
credit_account = xt_trader.query_credit_detail(acc)
if credit_account:
    credit_account_details=get_credit_account_details(credit_account)
    display(credit_account_details)
"""COMPACTS"""    
compacts = xt_trader.query_stk_compacts(acc)
if compacts:
    compact_details=get_compact_details(compacts)
    display(compact_details)

,订单编号,合同编号,证券代码,委托价格,成交价格,委托时间,委托状态,委托量,成交数量,已撤数量
0,1209008131,106,510330.SH,3.934,0.0,2023-05-12 02:08:52,ORDER_UNREPORTED,100,0,100
1,1209008133,107,510350.SH,4.139,0.0,2023-05-12 02:24:41,ORDER_CANCELED,100,0,100
2,1209008134,108,510330.SH,3.934,0.0,2023-05-12 02:24:41,ORDER_CANCELED,100,0,100
3,1209008129,94,510330.SH,3.934,0.0,2023-05-12 01:43:28,ORDER_CANCELED,200,0,200
4,1209008130,96,510330.SH,3.934,0.0,2023-05-12 01:54:49,ORDER_CANCELED,100,0,100


,账号类型,资金账号,可用金额,冻结金额,持仓市值,总资产
0,3,140080060269,5517.72,0.0,617.6,6535.72


,账号类型,资金账号,证券代码,持仓数量,可用数量,平均建仓成本,市值
0,3,140080060269,510330.SH,100,100,4.0100,393.4
1,3,140080060269,512800.SH,200,200,1.0855,224.2


,账号类型,资金账号,账号状态,更新时间,计算参数,冻结金额,总资产,可用金额,持仓盈亏,总市值,...,其它费用,融资授信额度,融资可用额度,融资冻结额度,融券授信额度,融券可用额度,融券冻结额度,融券卖出资金,已用融券卖出资金,剩余融券卖出资金
0,3,140080060269,0,1683820800,0,0.0,6535.72,5517.72,-0.5,617.6,...,0.0,1000000.0,999091.8,393.5,1000000.0,999091.8,0.0,400.4,0.0,400.4


,账号类型,资金账号,合约类型,头寸来源,证券市场,开仓日期,合约证券数量,未还合约数量,到期日,合约金额,合约息费,未还合约金额,未还合约息费,已还息费,已还金额,证券代码,合约编号,定位串
0,3,140080060269,融券,48,0,20230511,100,100,20231110,400.5,0.3,0.3,0.3,0.0,0.0,510330.SH,4172,20230511|00004172
1,3,140080060269,融资,48,0,20230510,100,0,20231109,114.1,0.1,114.2,0.1,0.0,0.0,512800.SH,4593,20230510|00004593


### Buy Portfolio
left symbol for long, right symbol for short.

### Short Portfolio

"""卖出 portfolio"""
order_id=short_portfolio(xt_trader,acc,symbols,hedge_ratio,captital,syncFlag=True)

### Cancel Orders with ORDER IDs

"""Cancel Orders"""
order_ids=[1209008133,1209008134]
for order in order_ids:
    cancel_result = xt_trader.cancel_order_stock(acc, order)
    if cancel_result == -1:
        print("Failed to cancel order.")
    else:
        print("Cancel order sucessfully.")

### 阻塞线程，接收交易推送

In [7]:
# xt_trader.run_forever()

In [8]:
xt_trader.stop()